### Stratégie :
On récupère le point de départ, on le région growth.
On fait une recherche sur la couche du dessous sur le centre trouvé précédemment puis en région growing sur les plus proches voisins jusqu'à trouver un point d'intensité similaire +-5% puis on le region growth. On limite le region growing à la taille de la tache trouvée sur la précédente couche + 1 voisin
Si on ne trouve rien dans cette zone, on met un False dans la liste et on descend d'une couche avec une tolérance d'intensité de +1% et d'un voisin supplémentaire 

In [13]:
import SimpleITK as sitk
import itk
import itkwidgets 
import matplotlib.pyplot as plt
from pathlib import Path
import numpy as np
import ipywidgets as widgets
import cv2
from tqdm.notebook import tqdm
from napari.utils.colormaps import colormap_utils as cu
from functools import partial

In [14]:

def regionSearch(image,point,searchRadius,minValue):
    search_memory = np.zeros_like(image, dtype=np.uint8)
    stack = [point]
    while stack:
        x, y = stack.pop()
        if search_memory[y, x] == 0:
            # Vérifier si la valeur du pixel est dans la plage acceptable
            # if point == (265,208):
            #     print(x, y)
            #     print(point)
            if int(image[y, x]) > minValue: #and int(image[y, x]) < maxValue:
                return (x,y)
            else:
                search_memory[y, x] = 1
            # Ajouter les voisins à la pile
            if abs(y-point[1]) < searchRadius and abs(x-point[0]) < searchRadius:
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        if 0 <= x + i < image.shape[1] and 0 <= y + j < image.shape[0]:
                            stack.append((x + i, y + j))
    print("Radius was "+str(searchRadius)+" origin was "+str(point) +" and value was "+str(minValue))
    return (-1,-1)

def regionGrowing(image,point,nested_intensity,alpha=0.60):
    segmented = np.zeros_like(image, dtype=np.uint8)

    # Initialiser la pile pour la croissance de la région
    stack = [point]

    # Récupérer la valeur du pixel du point de départ
    seed_value = nested_intensity.get_average()

    while stack:
        x, y = stack.pop()

        # Vérifier si le pixel est déjà visité
        if segmented[y, x] == 0:
            # Vérifier si la valeur du pixel est dans la plage acceptable
            if image[y, x] >= seed_value*alpha:
                # Ajouter le pixel à la région
                segmented[y, x] = 255

                # Ajouter les voisins à la pile
                for i in range(-1, 2):
                    for j in range(-1, 2):
                        if 0 <= x + i < image.shape[1] and 0 <= y + j < image.shape[0]:
                            stack.append((x + i, y + j))
    contours, _ = cv2.findContours(segmented, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)
    if len(contours)>0:
        return contours[0]
    else:
        print("Nothing found")
        return list()
    

In [15]:
class RunningAverage:
    def __init__(self,sum=0,count=0,last=None):
        self.sum = sum
        self.count = count
        self.last = last
        

    def add_number(self, number):
        self.sum += number
        self.last = number
        self.count += 1

    def get_average(self):
        if self.count == 0:
            return 0 
        return self.sum / self.count
    
    def split(self):
        self.sum /= 4
        self.count /= 2
        return self
    
    def get_last(self):
        return self.last
    
    def copy(self):
        return RunningAverage(self.sum,self.count,self.last)
    
class NestedAverage:
    def __init__(self,numbers=list(),limit=10,average=0,count=0):
        self.limit = limit
        self.average = average
        self.count = count
        self.numbers = numbers
        if self.average == 0 and self.count == 0:
            self.numbers = []
        
        
        
    def add_number(self, number):
        self.numbers.append(number)
        if self.count > self.limit:
            self.numbers.pop(0)
        else:
            self.count += 1
        self.average = sum(self.numbers)/self.count

    def get_average(self):
        return self.average
    
    def copy(self):
        return NestedAverage(self.numbers,self.limit,self.average,self.count)

class NetworkEngine:
    def __init__(self,image,max_depth):
        self.image = image
        self.network_manager = NetworkManager(max_depth)
        self.stack = list()
        self.branch_depth = [max_depth-1]
        self.current_depth = max_depth-1
        self.alpha = 0.6
        self.beta = 0.6
        self.pbar = tqdm(total=max_depth)

    def get_new_branch(self,parent_branch_id):
        child_id = self.network_manager.get_new_branch(parent_branch_id)
        self.branch_depth.append(self.branch_depth[parent_branch_id])
        return child_id
    def success_split(self,parent_branch,new_target):
        self.network_manager.set_branch_target(parent_branch,new_target)
        self.network_manager.get_mean_radius(parent_branch).split()

    def explore(self,branch_id,iteration=1,onsuccess=None):
        multi = True if iteration > 1 else False
        if multi:
            print("multi"+str(branch_id))
        for i in range(iteration):
            
            foundTarget = regionSearch(self.image[self.branch_depth[branch_id],:,:],self.network_manager.get_branch_target(branch_id),self.network_manager.get_mean_radius(branch_id).get_average(),self.network_manager.get_nested_intensity(branch_id).get_average()*self.alpha)
            tube_contour = regionGrowing(self.image[self.branch_depth[branch_id],:,:],foundTarget,self.network_manager.get_nested_intensity(branch_id))
            if len(tube_contour)==0:
                break
            center, radius = cv2.minEnclosingCircle(tube_contour)
            self.network_manager.get_mean_radius(branch_id).add_number(radius)
            self.network_manager.set_branch_target(branch_id,center)
            self.network_manager.get_nested_intensity(branch_id).add_number(self.image[self.branch_depth[branch_id],self.network_manager.get_branch_target(branch_id)[1],self.network_manager.get_branch_target(branch_id)[0]])
            self.network_manager.append_to_network(self.branch_depth[branch_id],branch_id,tube_contour)
            if not multi and radius <  self.network_manager.get_mean_radius(branch_id).get_average()*0.6:
                #We suspect a splitted vessel
                last_radius = self.network_manager.get_mean_radius(branch_id).get_last()
                x_offset = int(last_radius/4)
                pixel_1 = (self.network_manager.get_branch_target(branch_id)[0]-x_offset,self.network_manager.get_branch_target(branch_id)[1])
                pixel_2 = (self.network_manager.get_branch_target(branch_id)[0]+x_offset,self.network_manager.get_branch_target(branch_id)[1])
                #Creating a new branch with a new target
                target_2_branch_id = self.get_new_branch(branch_id)
                self.network_manager.set_branch_target(target_2_branch_id,pixel_2)
                #Exploring 20 frames of new branch with a success callback for parent branch
                self.stack.append([target_2_branch_id,20,partial(self.success_split, branch_id, pixel_1)])
            else:
                self.branch_depth[branch_id]-=1
        if multi and i == iteration-1:
            onsuccess()
        return None
        
    def explore_all(self):
        to_explore = [[i] for i in range(len(self.branch_depth)) if self.branch_depth[i] == self.current_depth]
        self.stack.extend(to_explore)

    def run(self):
        self.explore_all()
        while self.current_depth > 0:
            while len(self.stack) > 0:
                exploring_parameters = self.stack.pop()
                self.explore(*exploring_parameters)
                
            self.current_depth -= 1
            self.pbar.update(1)
            self.pbar.set_description("Searching region %s" % str(self.branch_depth))
            self.explore_all()
            

class NetworkManager:
    def __init__(self,depth):
        self.last_branch_id = 0
        self.network = list()
        for i in range(depth):
            self.network.append(list())
        self.branch_length = [0]
        self.branch_target = [0]
        self.branch_list = [0]
        self.branch_mean_radius = [RunningAverage()]
        self.branch_nested_intensity = [NestedAverage()]
        
    def get_new_branch(self,parent_branch_id=0):
        self.last_branch_id = self.last_branch_id+1
        self.branch_length.append(0)
        self.branch_target.append(0)
        self.branch_list.append(self.last_branch_id)
        self.branch_mean_radius.append(self.branch_mean_radius[parent_branch_id].split().copy())
        self.branch_nested_intensity.append(self.branch_nested_intensity[parent_branch_id].copy())
        return self.last_branch_id
    
    def get_mean_radius(self,branch_id):
        return self.branch_mean_radius[branch_id]

    def get_nested_intensity(self,branch_id):
        return self.branch_nested_intensity[branch_id]

    def get_branch_target(self,branch_id):
        return self.branch_target[branch_id]

    def set_branch_target(self,branch_id,target):
        self.branch_target[branch_id] = (int(target[0]),int(target[1]))

    def append_to_network(self,depth,branch_id,contour,predicted=False):
        self.network[depth].append(NetworkManagerArea(branch_id,contour,predicted))
        self.branch_length[branch_id]+=1
    
    def get_branchs(self):
        return self.branch_list

    def get_branch(self,branch_id):
        branch = list()
        offset = 0
        for i in reversed(range(len(self.network))):
            area = next((area for area in self.network[i] if area.branch_id == branch_id),None)
            if len(branch) == 0:
                offset = i
            if area != None:
                branch.append(area)
            else:
                if len(branch)>0:
                    break
        return branch,offset

    def get_branch_length(self,branch_id):
        return self.branch_length[branch_id]

    def remove_branch(self,branch_id):
        found = False
        for i in reversed(range(len(self.network))):
            area = next((area for area in self.network[i] if area.branch_id == branch_id),None)
            if area != None:
                found = True
                self.network[i].remove(area)
            else:
                #La branche a été trouvée et on est arrivée au bout de sa supression
                if found:
                    break
        self.branch_length[branch_id] = 0
        self.branch_list.remove(branch_id)
    
    #Each subnetwork is a different image
    def generate3DImages(self,shape):
        generated_images = []
        print(self.branch_list)
        print(self.branch_length)
        for branch_id in self.branch_list:
            image = np.zeros(shape,dtype=np.uint8)
            branch,offset = self.get_branch(branch_id)
            for area in branch:
                for point in area.contour:
                    image[offset][point[0][1]][point[0][0]] = 255
                offset -= 1
            generated_images.append(image)
        return generated_images

class NetworkManagerArea:
    def __init__(self,branch_id,contour,predicted):
        self.branch_id = branch_id
        self.contour = contour
        self.predicted = predicted

    def __str__(self) -> str:
        return "(branch_id : "+str(self.branch_id)+", contour : "+str(self.contour)+", predicted : "+str(self.predicted)+")"



In [16]:
def treeLookup(image,base_point,start=False,end=False,method="conic",alpha_search=0.6):
    #Processing default parameters
    if end == False:
        stop_index = 0
    else:
        stop_index = end

    if start == False:
        start_index = len(image)
    else:
        start_index = start
    mean_radius = RunningAverage()
    nested_intensity = NestedAverage()
    network_manager = NetworkManager(len(image))
    return subTreeLookup(network_manager,image,base_point,start_index,stop_index,method,alpha_search,mean_radius,nested_intensity,network_manager.get_new_branch())

def subTreeLookup(network_manager,image,base_point,start_index,stop_index,method,alpha_search,mean_radius,nested_intensity,branch_id):
    current_index = start_index
    current_pixel = base_point
    nested_intensity.add_number(image[start_index,base_point[1],base_point[0]])
    
    splitted_search = False
    successfull_split = False
    
    print("subtreeid "+str(branch_id)+" z-index "+str(current_index))
    tube_contour = regionGrowing(image[current_index,:,:],current_pixel,nested_intensity)
    if len(tube_contour) == 0:
        return network_manager
    center, radius = cv2.minEnclosingCircle(tube_contour)
    current_pixel = (int(center[0]),int(center[1]))
    mean_radius.add_number(radius)
    network_manager.append_to_network(current_index,branch_id,tube_contour)
    if branch_id == 1:
        pbar = tqdm(total=start_index-stop_index)
    
    while (current_index > stop_index - 1):
        current_index = current_index - 1
        if branch_id == 1:
            pbar.update(1)
        if splitted_search:
            if branch_id == 1:
                pbar.set_description("Searching for splitted in region %s" % current_index)
            last_radius = mean_radius.get_last()
            x_offset = int(last_radius/4)
            pixel_1 = (current_pixel[0]-x_offset,current_pixel[1])
            pixel_2 = (current_pixel[0]+x_offset,current_pixel[1])
            target_1 = regionSearch(image[current_index,:,:],pixel_1,(last_radius/2)-1,nested_intensity.get_average()*alpha_search)
            target_2 = regionSearch(image[current_index,:,:],pixel_2,(last_radius/2)-1,nested_intensity.get_average()*alpha_search)
            if target_1[0] != -1:
                print("target1 found "+str(target_1))
                foundTarget = target_1
            if target_2[0] != -1:
                target_2_branch_id = network_manager.get_new_branch()
                subTreeLookup(network_manager,image,target_2,current_index-1,stop_index,method,alpha_search,mean_radius.split().copy(),nested_intensity.copy(),target_2_branch_id)
                #TODO trouver un moyen de supprimer les sous reseaux trop court
                print("length :"+str(network_manager.get_branch_length(target_2_branch_id)))
                if network_manager.get_branch_length(target_2_branch_id) > 20:
                    successfull_split = True
                else:
                    network_manager.remove_branch(target_2_branch_id)
        else:
            #print("looking on xy:"+str(current_pixel)+" z:"+str(current_index)+" radius :"+str(radius))
            if branch_id == 1:
                pbar.set_description("Searching region %s" % current_index)
            foundTarget = regionSearch(image[current_index,:,:],current_pixel,radius,nested_intensity.get_average()*alpha_search)

        if foundTarget[0] != -1:
            if branch_id == 1:
                pbar.set_description("Growing region %s, radius %s" % (current_index,radius) )
            if splitted_search:
                tube_contour = regionGrowing(image[current_index,:,:],foundTarget,nested_intensity)
                if len(tube_contour) == 0:
                    print("target1 lead to end")
                    return network_manager
                center, radius = cv2.minEnclosingCircle(tube_contour)
            else:
                tube_contour = regionGrowing(image[current_index,:,:],foundTarget,nested_intensity)
                if len(tube_contour) == 0:
                    if network_manager.get_branch_length(branch_id) < 20:
                        network_manager.remove_branch(branch_id)  
                    return network_manager
                center, radius = cv2.minEnclosingCircle(tube_contour)
            
            if not successfull_split and radius < mean_radius.get_average()*0.6:
                current_index = current_index + 1
                splitted_search = True
                print("splitted suspicion : "+str(radius)+" while 60% mean "+str(mean_radius.get_average()*0.6))
                continue
            
            if successfull_split:
                mean_radius.split()
                print("splitted appended")
            successfull_split = False
            splitted_search = False

            mean_radius.add_number(radius)
            current_pixel = (int(center[0]),int(center[1]))
            nested_intensity.add_number(image[current_index,current_pixel[1],current_pixel[0]])
            network_manager.append_to_network(current_index,branch_id,tube_contour)
        else:
            return network_manager
            break
            #Target not found 
            
    # image_with_contours = image.copy()
    # cv2.drawContours(image_with_contours, contours, -1, 255, 10)
    # plt.imshow(image_with_contours)
    pbar.close()
    return network_manager



In [17]:
def subTreeLookupv2(network_manager,image,base_point,start_index,stop_index,method,alpha_search):
    current_index = start_index
    current_pixel = base_point
    network_manager.get_nested_intensity(0).add_number(image[start_index,base_point[1],base_point[0]])
    
    splitted_search = False
    successfull_split = False
    
    #print("subtreeid "+str(branch_id)+" z-index "+str(current_index))
    network_manager.set_branch_target(0,current_pixel)
    tube_contour = regionGrowing(image[current_index,:,:],network_manager.get_branch_target(0),network_manager.get_nested_intensity(0))
    if len(tube_contour) == 0:
        print("Can't grow any region, please verify your base_point and alpha values")
        return network_manager
    center, radius = cv2.minEnclosingCircle(tube_contour)
    network_manager.set_branch_target(0,center)
    network_manager.get_mean_radius(0).add_number(radius)
    network_manager.append_to_network(current_index,0,tube_contour)
    pbar = tqdm(total=start_index-stop_index)
    
    while (current_index > stop_index - 1):
        current_index = current_index - 1
        pbar.update(1)
        pbar.set_description("Searching region %s" % current_index)
        i = 0
        while i < len(NetworkManager.get_branchs()):
            branch_id = NetworkManager.get_branchs()[i]
            if splitted_search:
                pbar.set_description("Searching for splitted in region %s" % current_index)
                last_radius = network_manager.get_mean_radius(branch_id).get_last()
                x_offset = int(last_radius/4)
                pixel_1 = (network_manager.get_branch_target(branch_id)[0]-x_offset,network_manager.get_branch_target(branch_id)[1])
                pixel_2 = (network_manager.get_branch_target(branch_id)[0]+x_offset,network_manager.get_branch_target(branch_id)[1])
                target_1 = regionSearch(image[current_index,:,:],pixel_1,(last_radius/2)-1,network_manager.get_nested_intensity(branch_id).get_average()*alpha_search)
                target_2 = regionSearch(image[current_index,:,:],pixel_2,(last_radius/2)-1,network_manager.get_nested_intensity(branch_id).get_average()*alpha_search)
                if target_1[0] != -1:
                    foundTarget = target_1
                if target_2[0] != -1:
                    target_2_branch_id = network_manager.get_new_branch()
                    network_manager.set_branch_target(target_2_branch_id,target_2)
                    #TODO trouver un moyen de supprimer les sous reseaux trop court
            else:
                foundTarget = regionSearch(image[current_index,:,:],network_manager.get_branch_target(branch_id),radius,network_manager.get_nested_intensity(branch_id).get_average()*alpha_search)
                tube_contour = regionGrowing(image[current_index,:,:],foundTarget,network_manager.get_nested_intensity(branch_id))
                if len(tube_contour) == 0:
                    if network_manager.get_branch_length(branch_id) < 20:
                        network_manager.remove_branch(branch_id)  
                    return network_manager
                center, radius = cv2.minEnclosingCircle(tube_contour)
                network_manager.get_mean_radius(branch_id).add_number(radius)
                network_manager.set_branch_target(branch_id,center)
                network_manager.get_nested_intensity(branch_id).add_number(image[current_index,network_manager.get_branch_target(branch_id)[1],network_manager.get_branch_target(branch_id)[0]])
                network_manager.append_to_network(current_index,branch_id,tube_contour)
                #TODO reprendre ici et gérer le split en itératif
                if not successfull_split and radius <  network_manager.get_mean_radius(branch_id).get_average()*0.6:
                    i -= 1
                    splitted_search = True
                    print("splitted suspicion : "+str(radius)+" while 60% mean "+str(network_manager.get_mean_radius(branch_id).get_average()*0.6))
                    continue
            
            if foundTarget[0] != -1:
                if splitted_search:
                    tube_contour = regionGrowing(image[current_index,:,:],foundTarget,nested_intensity)
                    if len(tube_contour) == 0:
                        print(str(branch_id)+"stopped at z:"+str(current_index))
                        return network_manager
                    center, radius = cv2.minEnclosingCircle(tube_contour)
        else:
            return network_manager
            break
            #Target not found 
            
    # image_with_contours = image.copy()
    # cv2.drawContours(image_with_contours, contours, -1, 255, 10)
    # plt.imshow(image_with_contours)
    pbar.close()
    return network_manager



In [23]:
#TODO if validated split, region 1 can't hover region2 during the next 20 images
image_array = np.load(Path("temp/preprocessed_image.npy"))
network_engine = NetworkEngine(image_array,1532)
network_engine.network_manager.set_branch_target(0,(255,284))
network_engine.network_manager.get_nested_intensity(0).add_number(image_array[1531,284,255])
network_engine.network_manager.get_mean_radius(0).add_number(10)
network_engine.run()
# networkManager = treeLookup(image_array,(255,284),1531,300)

  0%|          | 0/1532 [00:00<?, ?it/s]

1531
1532
1530
1532
1529
1532
1528
1532
1527
1532
1526
1532
1525
1532
1524
1532
1523
1532
1522
1532
1521
1532
1520
1532
1519
1532
1518
1532
1517
1532
1516
1532
1515
1532
1514
1532
1513
1532
1512
1532
1511
1532
1510
1532
1509
1532
1508
1532
1507
1532
1506
1532
1505
1532
1504
1532
1503
1532
1502
1532
1501
1532
1500
1532
1499
1532
1498
1532
1497
1532
1496
1532
1495
1532
1494
1532
1493
1532
1492
1532
1491
1532
1490
1532
1489
1532
1488
1532
1487
1532
1486
1532
1485
1532
1484
1532
1483
1532
1482
1532
1481
1532
1480
1532
1479
1532
1478
1532
1477
1532
1476
1532
1475
1532
1474
1532
1473
1532
1472
1532
1471
1532
1470
1532
1469
1532
1468
1532
1467
1532
1466
1532
1465
1532
1464
1532
1463
1532
1462
1532
1461
1532
1460
1532
1459
1532
1458
1532
1457
1532
1456
1532
1455
1532
1454
1532
1453
1532
1452
1532
1451
1532
1450
1532
1449
1532
1448
1532
1447
1532
1446
1532
1445
1532
1444
1532
1443
1532
1442
1532
1441
1532
1440
1532
1439
1532
1438
1532
1437
1532
1436
1532
1435
1532
1434
1532
1433
1532
1432
1532


In [21]:
# mask = np.zeros_like(image_array, dtype=np.uint8)
# mask2 = np.zeros_like(image_array, dtype=np.uint8)
# last_search_mask = np.zeros_like(image_array, dtype=np.uint8)
# last_search_cut = last_search_mask[1265]
# cv2.circle(last_search_cut,(266,205), 361, 255, 1)
# cv2.circle(last_search_cut,(266,205), 1, 255, 1)
# last_search_mask[1532-len(tube_network)] = last_search_cut
#print(image_array.shape)
# index = 1531
# def draw_network(mask,mask2,tube_network,index,main=False):
#     for tube in tube_network:
#         #print("tube "+str(len(tube)))
#         if len(tube)>0:
#             if isinstance(tube[0][0],type(np.array([]))) and not isinstance(tube[0][0][0],type(np.intc(0))):
#                 mask,mask2 = draw_network(mask,mask2,tube,index)
#             else:
#                 for point in tube:
#                     if main:
#                         mask[index][point[0][1]][point[0][0]] = 255
#                     else:
#                         mask2[index][point[0][1]][point[0][0]] = 255
#             index=index-1
#     return mask,mask2
import napari
# mask,mask2 = draw_network(mask,mask2,tube_network,index,True)
mask_list = network_engine.network_manager.generate3DImages(image_array.shape)
viewer = napari.Viewer()
image_layer = viewer.add_image(image_array)
colorindex = 0
for mask in mask_list:
    image_layer = viewer.add_image(mask,colormap=list(cu.AVAILABLE_COLORMAPS.keys())[colorindex],blending="additive")
    colorindex+=1
    print(np.sum(mask))
#image_layer = viewer.add_image(mask2,colormap="blue",blending="additive")
#image_layer = viewer.add_image(last_search_mask,colormap="red",blending="additive")
# start the event loop and show the viewer
napari.run()
#print(sitk_image)


[0, 1, 2]
[269, 115, 23]
5703585
1233180
220575
